In [1]:
# !pip install transformers deepspeed-mii --upgrade

This enotebook follows the example given at https://github.com/microsoft/deepspeed-mii#deploying-mii-public

In [2]:
# no MII server is running
!sudo netstat -ltnp | grep -w ':50050' 

In [3]:
!nvidia-smi

Thu Nov 17 05:56:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   26C    P8    14W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import mii
mii_configs = {"tensor_parallel": 1, "dtype": "fp16"}
mii.deploy(task="text-generation",
           model="bigscience/bloom-560m",
           deployment_name="bloom560m_deployment",
           mii_config=mii_configs)

[2022-11-17 05:56:45,186] [INFO] [deployment.py:87:deploy] ************* MII is using DeepSpeed Optimizations to accelerate your model *************
[2022-11-17 05:56:45,252] [INFO] [server_client.py:219:_initialize_service] MII using multi-gpu deepspeed launcher:
 ------------------------------------------------------------
 task-name .................... text-generation 
 model ........................ bigscience/bloom-560m 
 model-path ................... /tmp/mii_models 
 port ......................... 50050 
 provider ..................... hugging-face 
 ------------------------------------------------------------
[2022-11-17 05:56:46,451] [WARNING] [runner.py:179:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2022-11-17 05:56:46,498] [INFO] [runner.py:508:main] cmd = /home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 

In [5]:
generator = mii.mii_query_handle("bloom560m_deployment")
result = generator.query({"query": ["DeepSpeed is", "Seattle is"]}, do_sample=True, max_new_tokens=30)

RuntimeError: This event loop is already running

In [6]:
!pip show transformers

Name: transformers
Version: 4.24.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache
Location: /home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, tokenizers, tqdm
Required-by: deepspeed-mii


In [7]:
!pip show deepspeed-mii

Name: deepspeed-mii
Version: 0.0.3
Summary: deepspeed mii
Home-page: http://deepspeed.ai
Author: DeepSpeed Team
Author-email: deepspeed-mii@microsoft.com
License: UNKNOWN
Location: /home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages
Requires: asyncio, deepspeed, grpcio, grpcio-tools, pydantic, torch, transformers
Required-by: 


In [ ]:
mii.terminate("bloom560m_deployment")